In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
loans_df = pd.read_csv('loan_status.csv')
loans_df.head()

,Loan_Status,Current_Loan_Amount,Term,Credit_Score,Annual_Income,Years_in_current_job,Home_Ownership,Purpose,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,Bankruptcies,Tax_Liens
0,Fully_Paid,99999999,Short_Term,741.0,2231892.0,8_years,Own_Home,Debt_Consolidation,29200.53,14.9,29.0,18,1,297996,750090.0,0.0,0.0
1,Fully_Paid,217646,Short_Term,730.0,1184194.0,<_1_year,Home_Mortgage,Debt_Consolidation,10855.08,19.6,10.0,13,1,122170,272052.0,1.0,0.0
2,Fully_Paid,548746,Short_Term,678.0,2559110.0,2_years,Rent,Debt_Consolidation,18660.28,22.6,33.0,4,0,437171,555038.0,0.0,0.0
3,Fully_Paid,99999999,Short_Term,728.0,714628.0,3_years,Rent,Debt_Consolidation,11851.06,16.0,76.0,16,0,203965,289784.0,0.0,0.0
4,Fully_Paid,99999999,Short_Term,740.0,776188.0,<_1_year,Own_Home,Debt_Consolidation,11578.22,8.5,25.0,6,0,134083,220220.0,0.0,0.0


In [2]:
# Generate our categorical variable list
loans_cat = loans_df.dtypes[loans_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
loans_df[loans_cat].nunique()

Loan_Status              2
Term                     2
Years_in_current_job    11
Home_Ownership           4
Purpose                  7
dtype: int64

In [3]:
# Check the unique value counts to see if binning is required
loans_df.Years_in_current_job.value_counts()

10+_years    13149
2_years       3225
3_years       2997
<_1_year      2699
5_years       2487
4_years       2286
1_year        2247
6_years       2109
7_years       2082
8_years       1675
9_years       1467
Name: Years_in_current_job, dtype: int64

In [4]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(loans_df[loans_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(loans_cat)
encode_df.head()

,Loan_Status_Fully_Paid,Loan_Status_Not_Paid,Term_Long_Term,Term_Short_Term,Years_in_current_job_10+_years,Years_in_current_job_1_year,Years_in_current_job_2_years,Years_in_current_job_3_years,Years_in_current_job_4_years,Years_in_current_job_5_years,...,Home_Ownership_Home_Mortgage,Home_Ownership_Own_Home,Home_Ownership_Rent,Purpose_Business_Loan,Purpose_Buy_House,Purpose_Buy_a_Car,Purpose_Debt_Consolidation,Purpose_Home_Improvements,Purpose_Medical_Bills,Purpose_Other
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [5]:
# Merge one-hot encoded features and drop the originals
loans_df = loans_df.merge(encode_df,left_index=True, right_index=True)
loans_df = loans_df.drop(loans_cat,1)
loans_df.head()

,Current_Loan_Amount,Credit_Score,Annual_Income,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,...,Home_Ownership_Home_Mortgage,Home_Ownership_Own_Home,Home_Ownership_Rent,Purpose_Business_Loan,Purpose_Buy_House,Purpose_Buy_a_Car,Purpose_Debt_Consolidation,Purpose_Home_Improvements,Purpose_Medical_Bills,Purpose_Other
0,99999999,741.0,2231892.0,29200.53,14.9,29.0,18,1,297996,750090.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,217646,730.0,1184194.0,10855.08,19.6,10.0,13,1,122170,272052.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,548746,678.0,2559110.0,18660.28,22.6,33.0,4,0,437171,555038.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,99999999,728.0,714628.0,11851.06,16.0,76.0,16,0,203965,289784.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,99999999,740.0,776188.0,11578.22,8.5,25.0,6,0,134083,220220.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [6]:
# Remove loan status target from features data
y = loans_df.Loan_Status_Fully_Paid
X = loans_df.drop(columns=["Loan_Status_Fully_Paid","Loan_Status_Not_Paid"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.849


In [8]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
854/854 [==============================] - 1s 601us/step - loss: 0.4708 - accuracy: 0.8041
Epoch 2/50
854/854 [==============================] - 1s 588us/step - loss: 0.3887 - accuracy: 0.8469
Epoch 3/50
854/854 [==============================] - 0s 572us/step - loss: 0.3871 - accuracy: 0.8453
Epoch 4/50
854/854 [==============================] - 1s 611us/step - loss: 0.3782 - accuracy: 0.8502
Epoch 5/50
854/854 [==============================] - 0s 565us/step - loss: 0.3795 - accuracy: 0.8494
Epoch 6/50
854/854 [==============================] - 0s 568us/step - loss: 0.3773 - accuracy: 0.8487
Epoch 7/50
854/854 [==============================] - 1s 658us/step - loss: 0.3765 - accuracy: 0.8496
Epoch 8/50
854/854 [==============================] - 1s 597us/step - loss: 0.3756 - accuracy: 0.8480
Epoch 9/50
854/854 [==============================] - 0s 565us/step - loss: 0.3803 - accuracy: 0.8461
Epoch 10/50
854/854 [==============================] - 1s 614us/step - loss: 0.370

In [9]:
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 24)                888       
_________________________________________________________________
dense_1 (Dense)              (None, 12)                300       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 13        
Total params: 1,201
Trainable params: 1,201
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [11]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
 86/854 [==>...........................] - ETA: 0s - loss: 0.3652 - accuracy: 0.8438  
Epoch 00001: saving model to checkpoints\weights.01.hdf5
160/854 [====>.........................] - ETA: 0s - loss: 0.3599 - accuracy: 0.8509
Epoch 00001: saving model to checkpoints\weights.01.hdf5
235/854 [=======>......................] - ETA: 0s - loss: 0.3591 - accuracy: 0.8533
Epoch 00001: saving model to checkpoints\weights.01.hdf5
396/854 [============>.................] - ETA: 0s - loss: 0.3599 - accuracy: 0.8549
Epoch 00001: saving model to checkpoints\weights.01.hdf5
473/854 [===============>..............] - ETA: 0s - loss: 0.3602 - accuracy: 0.8550
Epoch 00001: saving model to checkpoints\weights.01.hdf5
548/854 [==================>...........] - ETA: 0s - loss: 0.3604 - accuracy: 0.8550
Epoch 00001: saving model to checkpoints\weights.01.hdf5
621/854 [====================>.........] - ETA: 0s - loss: 0.3605 - accuracy: 0.8549
Epoch 00001: saving model to checkpoints\weights.

249/854 [=======>......................] - ETA: 0s - loss: 0.3498 - accuracy: 0.8596
Epoch 00007: saving model to checkpoints\weights.07.hdf5
320/854 [==========>...................] - ETA: 0s - loss: 0.3514 - accuracy: 0.8585
Epoch 00007: saving model to checkpoints\weights.07.hdf5
470/854 [===============>..............] - ETA: 0s - loss: 0.3531 - accuracy: 0.8574
Epoch 00007: saving model to checkpoints\weights.07.hdf5
550/854 [==================>...........] - ETA: 0s - loss: 0.3539 - accuracy: 0.8569
Epoch 00007: saving model to checkpoints\weights.07.hdf5
627/854 [=====================>........] - ETA: 0s - loss: 0.3545 - accuracy: 0.8564
Epoch 00007: saving model to checkpoints\weights.07.hdf5
702/854 [=======================>......] - ETA: 0s - loss: 0.3550 - accuracy: 0.8560
Epoch 00007: saving model to checkpoints\weights.07.hdf5
854/854 [==============================] - 1s 641us/step - loss: 0.3560 - accuracy: 0.8554
Epoch 8/100
  1/854 [..............................] - ET

387/854 [============>.................] - ETA: 0s - loss: 0.3526 - accuracy: 0.8582
Epoch 00013: saving model to checkpoints\weights.13.hdf5
550/854 [==================>...........] - ETA: 0s - loss: 0.3545 - accuracy: 0.8569
Epoch 00013: saving model to checkpoints\weights.13.hdf5
628/854 [=====================>........] - ETA: 0s - loss: 0.3551 - accuracy: 0.8565
Epoch 00013: saving model to checkpoints\weights.13.hdf5
702/854 [=======================>......] - ETA: 0s - loss: 0.3555 - accuracy: 0.8562
Epoch 00013: saving model to checkpoints\weights.13.hdf5
771/854 [==========================>...] - ETA: 0s - loss: 0.3557 - accuracy: 0.8561
Epoch 00013: saving model to checkpoints\weights.13.hdf5
854/854 [==============================] - 1s 662us/step - loss: 0.3559 - accuracy: 0.8559
Epoch 14/100
 90/854 [==>...........................] - ETA: 0s - loss: 0.3450 - accuracy: 0.8601
Epoch 00014: saving model to checkpoints\weights.14.hdf5
168/854 [====>.........................] - E

543/854 [==================>...........] - ETA: 0s - loss: 0.3585 - accuracy: 0.8561
Epoch 00019: saving model to checkpoints\weights.19.hdf5
715/854 [========================>.....] - ETA: 0s - loss: 0.3582 - accuracy: 0.8558
Epoch 00019: saving model to checkpoints\weights.19.hdf5
794/854 [==========================>...] - ETA: 0s - loss: 0.3580 - accuracy: 0.8557
Epoch 00019: saving model to checkpoints\weights.19.hdf5
854/854 [==============================] - 1s 647us/step - loss: 0.3580 - accuracy: 0.8555
Epoch 20/100
  1/854 [..............................] - ETA: 0s - loss: 0.1977 - accuracy: 0.9375
Epoch 00020: saving model to checkpoints\weights.20.hdf5
167/854 [====>.........................] - ETA: 0s - loss: 0.3664 - accuracy: 0.8459
Epoch 00020: saving model to checkpoints\weights.20.hdf5
245/854 [=======>......................] - ETA: 0s - loss: 0.3641 - accuracy: 0.8477
Epoch 00020: saving model to checkpoints\weights.20.hdf5
318/854 [==========>...................] - E

794/854 [==========================>...] - ETA: 0s - loss: 0.3571 - accuracy: 0.8548
Epoch 00025: saving model to checkpoints\weights.25.hdf5
854/854 [==============================] - 1s 648us/step - loss: 0.3571 - accuracy: 0.8548
Epoch 26/100
  1/854 [..............................] - ETA: 0s - loss: 0.3469 - accuracy: 0.8750
Epoch 00026: saving model to checkpoints\weights.26.hdf5
 78/854 [=>............................] - ETA: 0s - loss: 0.3615 - accuracy: 0.8540
Epoch 00026: saving model to checkpoints\weights.26.hdf5
225/854 [======>.......................] - ETA: 0s - loss: 0.3606 - accuracy: 0.8518
Epoch 00026: saving model to checkpoints\weights.26.hdf5
282/854 [========>.....................] - ETA: 0s - loss: 0.3592 - accuracy: 0.8526
Epoch 00026: saving model to checkpoints\weights.26.hdf5
425/854 [=============>................] - ETA: 0s - loss: 0.3574 - accuracy: 0.8539
Epoch 00026: saving model to checkpoints\weights.26.hdf5
481/854 [===============>..............] - E

...

383/854 [============>.................] - ETA: 0s - loss: 0.3637 - accuracy: 0.8531
Epoch 00075: saving model to checkpoints\weights.75.hdf5
451/854 [==============>...............] - ETA: 0s - loss: 0.3620 - accuracy: 0.8537
Epoch 00075: saving model to checkpoints\weights.75.hdf5
521/854 [=================>............] - ETA: 0s - loss: 0.3607 - accuracy: 0.8541
Epoch 00075: saving model to checkpoints\weights.75.hdf5
691/854 [=======================>......] - ETA: 0s - loss: 0.3587 - accuracy: 0.8546
Epoch 00075: saving model to checkpoints\weights.75.hdf5
766/854 [=========================>....] - ETA: 0s - loss: 0.3579 - accuracy: 0.8548
Epoch 00075: saving model to checkpoints\weights.75.hdf5
854/854 [==============================] - 1s 667us/step - loss: 0.3573 - accuracy: 0.8550
Epoch 76/100
  1/854 [..............................] - ETA: 0s - loss: 0.3752 - accuracy: 0.8750
Epoch 00076: saving model to checkpoints\weights.76.hdf5
 75/854 [=>............................] - E

568/854 [==================>...........] - ETA: 0s - loss: 0.3553 - accuracy: 0.8524
Epoch 00081: saving model to checkpoints\weights.81.hdf5
643/854 [=====================>........] - ETA: 0s - loss: 0.3544 - accuracy: 0.8529
Epoch 00081: saving model to checkpoints\weights.81.hdf5
717/854 [========================>.....] - ETA: 0s - loss: 0.3537 - accuracy: 0.8533
Epoch 00081: saving model to checkpoints\weights.81.hdf5
854/854 [==============================] - 1s 640us/step - loss: 0.3532 - accuracy: 0.8537
Epoch 82/100
  1/854 [..............................] - ETA: 1s - loss: 0.1982 - accuracy: 0.9375
Epoch 00082: saving model to checkpoints\weights.82.hdf5
 70/854 [=>............................] - ETA: 0s - loss: 0.3231 - accuracy: 0.8727
Epoch 00082: saving model to checkpoints\weights.82.hdf5
144/854 [====>.........................] - ETA: 0s - loss: 0.3329 - accuracy: 0.8656
Epoch 00082: saving model to checkpoints\weights.82.hdf5
318/854 [==========>...................] - E

709/854 [=======================>......] - ETA: 0s - loss: 0.3565 - accuracy: 0.8528
Epoch 00087: saving model to checkpoints\weights.87.hdf5
854/854 [==============================] - 1s 649us/step - loss: 0.3555 - accuracy: 0.8535
Epoch 88/100
  1/854 [..............................] - ETA: 0s - loss: 0.2378 - accuracy: 0.9375
Epoch 00088: saving model to checkpoints\weights.88.hdf5
 75/854 [=>............................] - ETA: 0s - loss: 0.3463 - accuracy: 0.8581
Epoch 00088: saving model to checkpoints\weights.88.hdf5
153/854 [====>.........................] - ETA: 0s - loss: 0.3484 - accuracy: 0.8549
Epoch 00088: saving model to checkpoints\weights.88.hdf5
226/854 [======>.......................] - ETA: 0s - loss: 0.3491 - accuracy: 0.8546
Epoch 00088: saving model to checkpoints\weights.88.hdf5
391/854 [============>.................] - ETA: 0s - loss: 0.3497 - accuracy: 0.8538
Epoch 00088: saving model to checkpoints\weights.88.hdf5
471/854 [===============>..............] - E

Epoch 94/100
  1/854 [..............................] - ETA: 0s - loss: 0.2052 - accuracy: 0.9688
Epoch 00094: saving model to checkpoints\weights.94.hdf5
167/854 [====>.........................] - ETA: 0s - loss: 0.3424 - accuracy: 0.8572
Epoch 00094: saving model to checkpoints\weights.94.hdf5
243/854 [=======>......................] - ETA: 0s - loss: 0.3463 - accuracy: 0.8557
Epoch 00094: saving model to checkpoints\weights.94.hdf5
317/854 [==========>...................] - ETA: 0s - loss: 0.3477 - accuracy: 0.8553
Epoch 00094: saving model to checkpoints\weights.94.hdf5
392/854 [============>.................] - ETA: 0s - loss: 0.3481 - accuracy: 0.8554
Epoch 00094: saving model to checkpoints\weights.94.hdf5
559/854 [==================>...........] - ETA: 0s - loss: 0.3490 - accuracy: 0.8554
Epoch 00094: saving model to checkpoints\weights.94.hdf5
638/854 [=====================>........] - ETA: 0s - loss: 0.3493 - accuracy: 0.8554
Epoch 00094: saving model to checkpoints\weights.9

245/854 [=======>......................] - ETA: 0s - loss: 0.3383 - accuracy: 0.8596
Epoch 00100: saving model to checkpoints\weights.100.hdf5
322/854 [==========>...................] - ETA: 0s - loss: 0.3390 - accuracy: 0.8595
Epoch 00100: saving model to checkpoints\weights.100.hdf5
395/854 [============>.................] - ETA: 0s - loss: 0.3397 - accuracy: 0.8594
Epoch 00100: saving model to checkpoints\weights.100.hdf5
469/854 [===============>..............] - ETA: 0s - loss: 0.3407 - accuracy: 0.8592
Epoch 00100: saving model to checkpoints\weights.100.hdf5
644/854 [=====================>........] - ETA: 0s - loss: 0.3434 - accuracy: 0.8581
Epoch 00100: saving model to checkpoints\weights.100.hdf5
724/854 [========================>.....] - ETA: 0s - loss: 0.3443 - accuracy: 0.8578
Epoch 00100: saving model to checkpoints\weights.100.hdf5
798/854 [===========================>..] - ETA: 0s - loss: 0.3449 - accuracy: 0.8576
Epoch 00100: saving model to checkpoints\weights.100.hdf5

In [16]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

KeyError: 0